# Litterbot
transfer the device from CPU memory to the GPU device

In [1]:
import torch
device = torch.device('cuda')

### Load TRT optimised models.
> IMPORTANT: Models must first be created with the notebooks for data collection and training. The models with the extension .pth must be in the same folder as this notebook. (Copy models into this folder)

In [2]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth')) # well trained road following model

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_collision_model_trt.pth')) # well trained collision avoidance model

<All keys matched successfully>

### Pre-Processing function
1. convert from HWC layout to CHW layout.
2. normalise with the same parameters as training (camera provides values in the range [0, 255] and the images loaded during training in the range [0, 1], so scaling by 255,0 is required. 
3. Transfer the data from the CPU memory to the GPU memory.
4. adding a batch dimension

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

### Starting the camera and creating the robot and GPS instance

In [4]:
from jetbot import Camera, bgr8_to_jpeg
import sys
import os
sys.path.append(os.path.abspath("/workspace/muellot_jetbot/jetbot"))
from robot2 import Robot2
from gps_coord import GPS_coords

camera = Camera.instance(width=224, height=224, fps=10)
robot = Robot2()
gps = GPS_coords()

### Kontrollinterface für den Roboter
Importing the required libraries

In [5]:
from IPython.display import display
import traitlets
from IPython.display import display
from ipywidgets import Layout, Label
import ipywidgets.widgets as widgets
import smopy

Button to get GPS-coordinates and print map

In [6]:
lout=Layout(align_items='center', border='3px solid black', padding='3px')

pos_button = widgets.Button(description='get Position')
map_widget = widgets.Image()
lat_widget = widgets.FloatText(value=0.0, description='Latitude :', disabled=True)
lon_widget = widgets.FloatText(value=0.0, description='Longitude:', disabled=True)
    
def get_map(change):
    gps.get_pos()
    lat_widget.value = gps.lat
    lon_widget.value = gps.lon
    maps = smopy.Map((lat_widget.value,lon_widget.value), z=18)
    x, y = maps.to_pixels(lat_widget.value,lon_widget.value)
    ax = maps.show_mpl(figsize=(8, 6));
    ax.plot(x, y, 'or', ms=10, mew=2);
    fig = ax.figure
    fig = fig.savefig('map.png')
    file = open("map.png", "rb")
    image = file.read()
    map_widget.value = image
    
pos_button.on_click(get_map)

mapnum_widget = widgets.VBox([pos_button,lat_widget,lon_widget])
mapshow_widget = widgets.HBox([mapnum_widget,map_widget], layout = Layout(align_items='center'))
map_display_widget = widgets.VBox([mapshow_widget], layout = lout)

Image widget

In [7]:
image_widget = widgets.Image()
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Stop-, Kill-, Auto- and Manual-Buttons

In [8]:
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')

stop_button = widgets.Button(description='stop', button_style='warning', layout=button_layout)
kill_button = widgets.Button(description='kill', button_style='danger', layout=button_layout)
sk_buttons = widgets.HBox([stop_button, kill_button])

state_widget = widgets.ToggleButtons(options=['Manu', 'Auto'], description='Operation', value='Manu')

buttons_box = widgets.VBox([sk_buttons, state_widget], layout = lout)

Sliders for adjustements, steering and throttle control

In [9]:
#Road Following sliders
network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=0.0, max=1.0, value=0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_dgain_slider = widgets.FloatSlider(description='Steering kd', min=0.0, max=0.5, step=0.001, value=0.06, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='Throttle', min=0.0, max=1.0, value=0.0, step=0.01, orientation='vertical')

#Collision Avoidance sliders
blocked_slider = widgets.FloatSlider(description='Blocked', min=0.0, max=1.0, orientation='horizontal')
stopduration_slider= widgets.IntSlider(description='Time for Stop', min=1, max=1000, step=1, value=10, orientation='horizontal') 
blocked_threshold= widgets.FloatSlider(description='Blocked Threshold', min=0, max=1.0, step=0.01, value=0.8, orientation='horizontal')
block_button = widgets.Button(description='free', button_style='success', disabled=True, layout=button_layout)

Link values from sliders to robot class

In [10]:
steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (robot, 'steering_gain'))
steering_value_link  = traitlets.link((steering_value_slider, 'value'), (robot, 'steering'))
throttle_slider_link = traitlets.link((throttle_slider, 'value'), (robot, 'throttle'))

Build Boxes to contain Sliders, Buttons an live Image

In [11]:
roadF_widgets = widgets.VBox([Label('Road Following Sliders'),
            network_output_slider,
            steering_gain_slider,
            steering_dgain_slider,
            steering_value_slider], 
            layout = lout)

roadC_widgets = widgets.VBox([Label('Collision Avoidance Sliders'),
            blocked_slider,
            stopduration_slider,
            blocked_threshold],
            layout = lout)

roadFundC_widgets = widgets.HBox([roadF_widgets, throttle_slider, image_widget, block_button, roadC_widgets])

auto_widgets = widgets.VBox([Label('Automatic Mode'), roadFundC_widgets], layout = lout)

mcon_widgets = widgets.HBox([
    steering_value_slider,
    throttle_slider,
    image_widget],
    layout = lout)

manu_widgets = widgets.VBox([Label('Manual Mode'), mcon_widgets], layout = lout) 

### Execute Function
This is where the magic happens :)

function that will get called whenever the camera's value changes. This function will do the following steps:

1. Pre-process the camera image
2. Execute the neural network models for Road following and Collision Avoidance
3. Check an if statements which would allow the Jetbot to perform road following and stop whenever an obstacle has been detected
4. Compute the approximate steering value
5. Control the motors using proportional / derivative control (PD)

In [12]:
import cv2
import math

angle = 0.0
angle_last = 0.0
count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
x = 0.0
y = 0.0
speed_value = throttle_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
       
    image_preproc = preprocess(change['new']).to(device)
     
    #Collision Avoidance model:
    
    prob_blocked = float(F.softmax(model_trt_collision(image_preproc), dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
            block_button.button_style = 'danger'
            block_button.description = 'blocked'
        else:
            #start of road following detection
            block_button.button_style = 'success'
            block_button.description = 'free'
            go_on = 1
            count_stops = 0
            xy = model_trt(image_preproc).detach().float().cpu().numpy().flatten()        
            x = xy[0]            
            y = (0.5 - xy[1]) / 2.0
            network_output_slider.value = x
            speed_value = throttle_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            x = 0.0 #set x steering to zero
            y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero
        else:
            go_on = 1
            count_stops = 0
            
    
    angle = math.atan2(x, y)        
    pid = angle * steer_gain + (angle - angle_last) * steer_dgain
    steering_value_slider.value = pid
    angle_last = angle
    robot.throttle = speed_value
    robot.steering = max(min(steering_value_slider.value, 1.0), -1.0) 

execute({'new': camera.value}) # call the function once to initialize
robot.stop()

### Interface
Link functions to buttons to use them (Start and Stop of robot, switching between manual and automatic mode, get GPS-coordinates)
>WARNING: If toggle button is switched from  manual to auto the robot will move, if throttle ist not zero. When the kill button is pressed, the notebook must be restarted.

Below this cell the interface for interaction with the robot should appear
>By right-clicking on the interface, you can display it in a separate output view.

In [13]:
import time

def stop(change):
    robot.stop()
    state_widget.value = 'Manu'
    
stop_button.on_click(stop)

def kill(change):
    robot.stop()
    camera.stop()
    
kill_button.on_click(kill)
        
def auto_manu_changed(change):
    if (state_widget.value == 'Manu'):
        camera.unobserve(execute, names='value')
        time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
        robot.stop()
    elif (state_widget.value == 'Auto'):
        camera.observe(execute, names='value')

state_widget.observe(auto_manu_changed, 'value')

display(buttons_box)
display(manu_widgets)
display(auto_widgets)
display(map_display_widget)

### Connection of Controller
Activate manual operation using a controller for more comfortable manual operation. Simply uncomment the code in the cells below.

In [18]:
"""
controller = widgets.Controller(index=1)  # replace with index of your controller

display(controller)
"""

Controller(index=1)

In [19]:
"""
cont_steer_link = traitlets.dlink((controller.axes[0], 'value'), (robot, 'steering'))
cont_throt_link = traitlets.dlink((controller.axes[3], 'value'), (robot, 'throttle'), transform=lambda x: -x)
"""